In [45]:
import numpy as np
import os
import glob
import cv2
import datetime

In [125]:
# ok so we start with two vides from cameras
# 

def load_times(fname,
               minute):
    data = np.load(fname, allow_pickle=True)
    #print (data.files)
    frame_times_ms = data['frame_times']//1000
    #print (frame_times_ms)
    recording_start_time = data['recording_start_time']
    #print ("recording start time: ", recording_start_time)
    encoder_start = data['encoder_start']
    #print ("encoder start: ", encoder_start)

    ###############################################
    # convert july 24, 2025  exacdtly midngith to milisecond in epoch systm eimte clock but make sure its' UTC+1 london time
    #epoch_start = np.datetime64('2025-07-24T00:00:00', 'ms') - np.timedelta64(1, 'h')  # UTC+1
    #epoch_start_ms = epoch_start.astype('datetime64[ms]').astype(int)//1000

    # so this is the video time stamps relative to the epoch start
    #delta_times = frame_times_ms - epoch_start_ms
    #print("time srelative to midnight (in ms): ", delta_times)

    # now convert into bucket discrete time
    time_stamps_binned = frame_times_ms // 10 * 10 # convert to millseconds and round to 10ms bin
    #print("time relative to midnight in 10ms buckets: ", delta_times_bucket)

    # and convert into a discrete bin of 10ms from midnight
    #delta_times_bucket_discrete = delta_times_bucket // 10
    #print("time relative to midnight in 10ms discrete buckets: ", delta_times_bucket_discrete)

    return time_stamps_binned

#
def decompress_video(minute, 
                     fname,
                     root_dir,
                     cam,
                     time_stamps_binned,
                     unix_time_to_start_of_minute,
                     shrink_factror=1,
                     overwrite_existing=False):
    
    ''' use OpenCV or something fast to load the video file or uncompress it.
        Not clear yet if we should just save to disk, but probably,
    '''
    # use opencv to load video
    # and save the png files with the filenmae of 
    

    frame_height = 720
    frame_width  = 1280
    channels = 3
    # shrink based on the shrink factor
    if shrink_factror > 1:
        frame_height //= shrink_factror
        frame_width  //= shrink_factror
        print ("shrinking video frames to: ", frame_height, "x", frame_width)
    frame_size_bytes = frame_height * frame_width * channels
    blank_frame = np.zeros((frame_height, frame_width, channels), dtype=np.uint8)

    # ok so we now have to take the time stamps 
    times_relative_to_min_start = time_stamps_binned - unix_time_to_start_of_minute
    print ("time stamps relative to start of minute: ", times_relative_to_min_start)
    print (" total frames: ", len(times_relative_to_min_start), 
           "total unique frames:", np.unique(times_relative_to_min_start).shape[0])

    # make file names
    fname_video_current_minute = os.path.split(fname)[0]+"/minute_" + str(minute) + ".bin"
    fname_video_next_minute = os.path.split(fname)[0]+"/minute_" + str(minute+1) + ".bin"

    # Step 1: Check if file exists & count existing frames
    # then we need to indepnt to existin frame to start appending
    if os.path.exists(fname_video_current_minute):
        file_size = os.path.getsize(fname_video_current_minute)
        frames_already_written = file_size // frame_size_bytes
        print(f"File already exists: {frames_already_written} frames found.")
    else:
        frames_already_written = 0
        print("File does not exist yet.")

    # need to check that if there is a full video in place already then we can exit safely
    if frames_already_written >= 6000:
        print ("... this minute of data already decompressed, exiting ...")
        return

    # we need to make sure that the first time stamp is 0

    # so we now loop over the video and the raw frames 
    # we fill blanks until we reach the next time stamp

    # use open cv to laod video
    import cv2
    print ("opening video file: ", fname)
    cap = cv2.VideoCapture(fname)
    if not cap.isOpened():
        raise IOError("Cannot open video file: {}".format(fname))

    #
    ctr_frame = 0
    ctr_bin = frames_already_written*10
    n_frames_read = 0
    n_unique_frames_written = 0
    with open(fname_video_current_minute, 'ab') as f:
        
        ##################### FILL EXISTING VIDEO WITH BLANKS ######################
        # if video already in place need to then fill the gap between the previous video ended
        #  and current one; usually about 3-5 seconds of duration during file upload to the server
        if frames_already_written > 0:
            print ("... video exists filling up downtime with blank frames ...")
            # TODO: fill in video with last frame - so we load it
            while ctr_bin != times_relative_to_min_start[ctr_frame]:
                # we write on frame to stack and advance 10 ms
                f.write(blank_frame.tobytes())
                ctr_bin += 10
                # print ("ctr_bin: ",ctr_bin)
                # print ("ctr_frame: ", ctr_frame)
                # print ("times_relative to min start: ", times_relative_to_min_start[ctr_frame])
                # don't increment the video frame coutner; 
                # we're just trying to catch up to it with the blank frames

        ####################################################################
        ##################### LOOP OVER 60 sec VIDEO  ######################
        ####################################################################
        while ctr_bin < 60000:
            ret, frame = cap.read()
            if not ret:
                break

            # let's print size of the frame
            #print ("frame shape: ", frame.shape, " frame size: ", frame.size, " bytes: ", frame.nbytes)

            #return

            #if ctr_bin%5000==0:
            #    print ("processing frame: ", ctr_bin, " / ", 60000, " frames written: ", n_frames_written)

            # we subsample/shrink frame based on shrink factor
            if shrink_factror > 1:
                # let's use simple subsampling
                if True:
                    frame = frame[::shrink_factror, ::shrink_factror, :]
                else:
                    #
                    frame = cv2.resize(frame, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)

            # 
            while ctr_bin < times_relative_to_min_start[ctr_frame]:
                # we need to write a blank frame
                f.write(blank_frame.tobytes())
                ctr_bin += 10  # increment the bin counter
                # increment the bin counter

            # write the frame to the binary file
            #print ("ctr_bin", ctr_bin, ", ctr_frame: ", ctr_frame, times_relative_to_min_start[ctr_frame])
            n_frames_read += 1

            # check to makes sure next value is different:
            # we only write frames that are unique
            # don't generallyneed to check if we have more ctr_frame than 
            if (ctr_frame+1)<=times_relative_to_min_start.shape[0]:
                if times_relative_to_min_start[ctr_frame]!=times_relative_to_min_start[ctr_frame+1]:
                    f.write(frame.tobytes())
                    ctr_bin += 10
                    n_unique_frames_written += 1

            #
            ctr_frame += 1

            # we also replace the blank frame now with the last read frame
            blank_frame = frame.copy()

    #
    print ("# UNIQUE frames written current min", n_unique_frames_written,
           ", n_frames_read: ", n_frames_read)
    print ("last frame time written: ", times_relative_to_min_start[ctr_frame], " ctr_frame: ", ctr_frame)

    #########################################################################
    ################# Extact frames from 2nd minute #########################
    #########################################################################
    # let's clip the frames relative to start of 2nd miunte so basically ctr_frame
    times_relative_to_min_start = times_relative_to_min_start[ctr_frame:] - 60000 +10
    print ("writing second minute frames starting at: ", times_relative_to_min_start[:5])

       # write the rest of the frames to the next vid
    ctr_bin = 0 
    n_frames_read = 0
    n_unique_frames_written = 0
    ctr_frame = 0

    # Step 2: Open in append mode ('ab' = append binary)
    with open(fname_video_next_minute, 'wb') as f:

        #
        while ctr_bin < 60000:
            ret, frame = cap.read()
            if not ret:
                #print ("... NO MORE VID FRAMES...")
                break
            #
            #if ctr_bin%5000==0:
            #    print ("processing frame: ", ctr_bin, " / ", 60000, " frames written: ", n_frames_written)

            # apply shrink factor
            if shrink_factror > 1:
                # let's use simple subsampling
                if True:
                    frame = frame[::shrink_factror, ::shrink_factror, :]
                else:
                    #
                    frame = cv2.resize(frame, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)

            # 
            #print ("ctr_bin", ctr_bin, ", ctr_frame: ", ctr_frame, times_relative_to_min_start[ctr_frame])
            while ctr_bin != times_relative_to_min_start[ctr_frame]:
                # we need to write a blank frame
                f.write(blank_frame.tobytes())
                ctr_bin += 10  # increment the bin counter
                # increment the bin counter

            #
            if (ctr_frame+1)<=(times_relative_to_min_start.shape[0]-1):
                if times_relative_to_min_start[ctr_frame]!=times_relative_to_min_start[ctr_frame+1]:
                    f.write(frame.tobytes())
                    ctr_bin += 10
                    n_unique_frames_written += 1

            # write the frame to the binary file
            n_frames_read += 1

            # check to makes sure next value is different:
            if ctr_frame+1 >= len(times_relative_to_min_start):
                #print ("... NO MORE TIME STAMPS...")
                break


            #
            ctr_frame += 1
            
            # we also replace the blank frame now with the last read frame
            blank_frame = frame.copy()

    print ("# UNIQUE frames written to next min video", n_unique_frames_written,
              ", n_frames_read: ", n_frames_read)

#
def make_video(root_dir,
                minute,
                n_cams,
                fname_combined,
                shrink_factor=1,
                overwrite_existing = False
                ):
    
    from tqdm import trange
    ''' make a video from the available frames for this minute '''
    
        
    frame_height = 720        # pixels
    frame_width = 1280       # pixels
    channels = 3               # RGB
    if shrink_factor > 1:
        frame_height //= shrink_factor
        frame_width  //= shrink_factor
        print ("shrinking video frames to: ", frame_height, "x", frame_width)
    frame_size_bytes = frame_height * frame_width * channels  # 1024 * 768 * 3 = 2,359,296 bytes

    if os.path.exists(fname_combined) and overwrite_existing==False:
        print ("... combined video file already exists: ", fname_combined)
        return

    rows, cols = 3, 6
    frame_all_cams_blank = np.zeros((frame_height * rows, frame_width * cols, channels), dtype=np.uint8)

    print ("creating combined video file: ", fname_combined)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(fname_combined, fourcc, 30.0, (frame_width * cols, frame_height * rows))

    # we actually want to grab freeze frames from non-recorded cameras - so we don't reinitialize this
    for i in trange(6000):
    
        # loop over cameras and grab a frame from each
        for cam in range(1,n_cams+1,1):
            # find the filename for this camera and bin
            fname_frame = os.path.join(root_dir,
                                    str(cam),
                                    "minute_"+
                                    str(minute) + ".bin")
            
            #
            if os.path.exists(fname_frame):
                # we need to index into this file to the current frame i using framesize-bytes
                with open(fname_frame, 'rb') as f:
                    f.seek(i * frame_size_bytes)

                    frame_data = f.read(frame_size_bytes)
                    if len(frame_data) != frame_size_bytes:
                        # leave the previous frame in place
                        continue
                    frame = np.frombuffer(frame_data, dtype=np.uint8).reshape((frame_height, frame_width, channels))

                    try:
                        frame = np.frombuffer(frame, dtype=np.uint8).reshape((frame_height, 
                                                                          frame_width, 
                                                                          channels))
                    except:
                        # ok we need to print a lot of metadata to understan why we're reading patst the end of the file
                        print ("i: ", i)
                        print ("cam: ", cam)
                        print ("fname: ", fname_frame)
                        print ("frame_size_bytes: ", frame_size_bytes)

                # 
                # adding frame from camera:  2  to row:  2  col:  0  at position:  144 216 0 128
                # adding frame from camera:  3  to row:  0  col:  0  at position:  0 72 0 128
                                
                col = 5 - ((cam - 1) // 3)  # 6 total columns → index 0..5 reversed
                row = (cam - 1) % 3  # 0=top, 1=middle, 2=bottom in OpenCV coords

                r0 = row * frame_height
                r1 = r0 + frame_height
                c0 = col * frame_width
                c1 = c0 + frame_width

                frame_all_cams_blank[r0:r1, c0:c1, :] = frame


                #
        #         print ("adding frame from camera: ", cam, 
        #                " to row: ", row, " col: ", col, 
        #                " at position: ", r0, r1, c0, c1)

        # return
            
        # now write the combined frame to the video file
        out.write(frame_all_cams_blank)
    # this is probably coming at the end. not sure exactly
    out.release()
    print ('finished writing combined video file: ', fname_combined)


In [ ]:
from datetime import datetime, timezone, timedelta

#############################################
#############################################
#############################################
n_cams = 18
root_dir = "/home/cat/Downloads/data_stitching/cams/"
date = "2025_07_31"                               # date of the video
hour_start = "07"                                     # start at midnight
n_mins = 60
shrink_factor = 10   # this shrinks the video along x and y axis by this factor; for now we use subsampling
cage_id = 1

# outer loop gooin over every minue of time 
for minute in range(n_mins):

    # save the combined frame as a video file
    fname_combined = os.path.join(root_dir, "minute_" +str(minute)+'.avi')

    if os.path.exists(fname_combined):
        print ("Video exists for Minute ", str(minute), "...skipping...")
        print ('')
        print ('')
        continue

    # find the filename for each camera that falls in minute #1
    for cam in range(1,n_cams+1,1):
        fname_root = os.path.join(root_dir, str(cam), 
                                  str(cage_id) + "_" +
                                  str(cam) + "_" +
                                  date+"_"+str(hour_start)+"_"+str(minute).zfill(2)+"*.npz")

        #print ("fname root: ", fname_root)
        # find any filenmes that match this pattern
        # There can only be 0 or 1 videos at most as we save 1 minute video per camera
        fnames = glob.glob(fname_root)

        # if the camera has files we then need to load the frame times
        if len(fnames)==0:
            print ('... no files found for camera: ', cam, " minute: ", minute)
            print ('')
            continue           
        print ("minute:", minute, " cam:", cam, " files:", fnames)

        # load the time stamps for this camera
        time_stamps_binned = load_times(fnames[0], minute)
        print ("time stamps binned: ", time_stamps_binned)

        # Convert to UTC+1 using timezone offset
        dt_naive = datetime.strptime(f"{date.replace('_', '-')} {hour_start}:{minute:02d}", "%Y-%m-%d %H:%M")
        dt_utc1 = dt_naive.replace(tzinfo=timezone(timedelta(hours=1)))  # UTC+1

        # Get absolute time in nanoseconds
        timestamp_ns = int(dt_utc1.timestamp() * 1_000_000_000)

        # Truncate to milliseconds
        unix_time_to_start_of_minute = timestamp_ns // 1_000_000
        print("absolute unix time (ms, UTC+1) to start of minute:", unix_time_to_start_of_minute)

        #################################################
        # process #1
        fname_video = fnames[0].replace('_metadata.npz', '.h264') 
        # use opencv to uncomrpess the video to .png files on disk
        decompress_video(minute,
                         fname_video,
                         root_dir,
                         cam,
                         time_stamps_binned,
                         unix_time_to_start_of_minute, 
                         shrink_factor,
                         overwrite_existing=True)     
        print ('')

    ######################################
    # process #2 - here we make the mosaic 1 minute video based on the available files
    # we loop over all possible files 

    make_video(root_dir,
               minute,
               n_cams,
               fname_combined,
               shrink_factor=shrink_factor,
               overwrite_existing=True)
    
    print ("***************************")
    print ('')

    

... no files found for camera:  1  minute:  0

minute: 0  cam: 2  files: ['/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_00_41_557_metadata.npz']
time stamps binned:  [1753941641540 1753941641550 1753941641560 ... 1753941701520 1753941701530
 1753941701540]
absolute unix time (ms, UTC+1) to start of minute: 1753941600000
shrinking video frames to:  18 x 32
time stamps relative to start of minute:  [ 41540  41550  41560 ... 101520 101530 101540]
 total frames:  7204 total unique frames: 6001
File does not exist yet.
opening video file:  /home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_00_41_557.h264
# UNIQUE frames written current min 1846 , n_frames_read:  2216
last frame time written:  60000  ctr_frame:  2216
writing second minute frames starting at:  [10 20 30 30 40]
# UNIQUE frames written to next min video 4154 , n_frames_read:  4987

minute: 0  cam: 3  files: ['/home/cat/Downloads/data_stitching/cams/3/1_3_2025_07_31_07_00_34_880_metadata.npz']
time stamps

100%|██████████| 6000/6000 [00:01<00:00, 4078.22it/s]


finished writing combined video file:  /home/cat/Downloads/data_stitching/cams/minute_0.avi
***************************

... no files found for camera:  1  minute:  1

minute: 1  cam: 2  files: ['/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_01_44_905_metadata.npz']
time stamps binned:  [1753941704890 1753941704900 1753941704910 ... 1753941764870 1753941764880
 1753941764890]
absolute unix time (ms, UTC+1) to start of minute: 1753941660000
shrinking video frames to:  18 x 32
time stamps relative to start of minute:  [ 44890  44900  44910 ... 104870 104880 104890]
 total frames:  7204 total unique frames: 6001
File already exists: 4155 frames found.
opening video file:  /home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_01_44_905.h264
... video exists filling up downtime with blank frames ...
# UNIQUE frames written current min 1511 , n_frames_read:  1814
last frame time written:  60000  ctr_frame:  1814
writing second minute frames starting at:  [10 20 30 40 40]


100%|██████████| 6000/6000 [00:01<00:00, 4100.29it/s]


finished writing combined video file:  /home/cat/Downloads/data_stitching/cams/minute_1.avi
***************************

... no files found for camera:  1  minute:  2

minute: 2  cam: 2  files: ['/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_02_48_483_metadata.npz']
time stamps binned:  [1753941768460 1753941768470 1753941768480 ... 1753941828460 1753941828470
 1753941828470]
absolute unix time (ms, UTC+1) to start of minute: 1753941720000
shrinking video frames to:  18 x 32
time stamps relative to start of minute:  [ 48460  48470  48480 ... 108460 108470 108470]
 total frames:  7205 total unique frames: 6002
File already exists: 4490 frames found.
opening video file:  /home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_02_48_483.h264
... video exists filling up downtime with blank frames ...
# UNIQUE frames written current min 1154 , n_frames_read:  1385
last frame time written:  60000  ctr_frame:  1385
writing second minute frames starting at:  [10 20 30 40 40]


...

100%|██████████| 6000/6000 [00:02<00:00, 2735.14it/s]


finished writing combined video file:  /home/cat/Downloads/data_stitching/cams/minute_11.avi
***************************

... no files found for camera:  1  minute:  12

minute: 12  cam: 2  files: ['/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_12_20_309_metadata.npz']
time stamps binned:  [1753942340290 1753942340300 1753942340310 ... 1753942400280 1753942400290
 1753942400300]
absolute unix time (ms, UTC+1) to start of minute: 1753942320000
shrinking video frames to:  18 x 32
time stamps relative to start of minute:  [20290 20300 20310 ... 80280 80290 80300]
 total frames:  7204 total unique frames: 6002
File already exists: 1677 frames found.
opening video file:  /home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_12_20_309.h264
... video exists filling up downtime with blank frames ...
# UNIQUE frames written current min 3971 , n_frames_read:  4767
last frame time written:  60000  ctr_frame:  4767
writing second minute frames starting at:  [10 20 30 40 50]
# U

100%|██████████| 6000/6000 [00:01<00:00, 3478.01it/s]


finished writing combined video file:  /home/cat/Downloads/data_stitching/cams/minute_12.avi
***************************

minute: 13  cam: 1  files: ['/home/cat/Downloads/data_stitching/cams/1/1_1_2025_07_31_07_13_01_267_metadata.npz']
time stamps binned:  [1753942381250 1753942381260 1753942381270 ... 1753942441240 1753942441250
 1753942441250]
absolute unix time (ms, UTC+1) to start of minute: 1753942380000
shrinking video frames to:  18 x 32
time stamps relative to start of minute:  [ 1250  1260  1270 ... 61240 61250 61250]
 total frames:  7204 total unique frames: 6001
File does not exist yet.
opening video file:  /home/cat/Downloads/data_stitching/cams/1/1_1_2025_07_31_07_13_01_267.h264
# UNIQUE frames written current min 5875 , n_frames_read:  7052
last frame time written:  60000  ctr_frame:  7052
writing second minute frames starting at:  [10 10 20 30 40]
# UNIQUE frames written to next min video 125 , n_frames_read:  151

minute: 13  cam: 2  files: ['/home/cat/Downloads/data_st

KeyboardInterrupt: 

In [ ]:
fname = '/home/cat/Downloads/data_stitching/1/minute_1.bin'

# let's load this using mmap as a binary file
data = np.memmap(fname, dtype=np.uint8, mode='r', shape=(6000, 720, 1280, 3))

# let's check the shape of the data
print("Data shape:", data.shape)

# ok now let's grab a frame and display it
idx = 2000
frame = data[idx]

# let's display the frame using opencv
cv2.imshow("Frame", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


Data shape: (6000, 720, 1280, 3)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/cat/miniconda3/lib/python3.11/site-packages/cv2/qt/plugins"


In [ ]:
# Ignore - past here

In [110]:
fname0 = '/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_00_41_557_metadata.npz'
fname2 = '/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_01_44_905_metadata.npz'

frame_times1 = np.load(fname1)['frame_times']
frame_times2 = np.load(fname2)['frame_times']
print ((frame_times1 - 1753941704894859)//1000+49880, "msecs")
print ((frame_times2 - 1753941704894859)//1000+49880-60000, "msecs")


[ 49880  49888  49896 ... 109864 109872 109881] msecs
[-10120 -10112 -10104 ...  49864  49872  49881] msecs


In [109]:
fname_bin1 = '/home/cat/Downloads/data_stitching/cams/2/minute_1.bin'
data_bin1 = np.fromfile(fname_bin1).reshape(32,18,3)
# print size of file
print (data_bin1.size/32/18)

print (data_bin1.shape)

ValueError: cannot reshape array of size 897480 into shape (32,18,3)

In [ ]:


#########################################
# LOAD video time stamps
data = np.load(fnames_cam1[0], allow_pickle=True)
print (data.files)
frame_times_ms = data['frame_times']//1000
print (frame_times_ms)
recording_start_time = data['recording_start_time']
print ("recording start time: ", recording_start_time)
encoder_start = data['encoder_start']
print ("encoder start: ", encoder_start)

###############################################
# convert july 24, 2025  exacdtly midngith to milisecond in epoch systm eimte clock but make sure its' UTC+1 london time
epoch_start = np.datetime64('2025-07-24T00:00:00', 'ms') - np.timedelta64(1, 'h')  # UTC+1
epoch_start_ms = epoch_start.astype('datetime64[ms]').astype(int)

# so this is the video time stamps relative to the epoch start
delta_times = frame_times_ms - epoch_start_ms
print("time srelative to midnight (in ms): ", delta_times)

# now convert into bucket discrete time
delta_times_bucket = delta_times // 10 * 10 # convert to seconds
print("time relative to midnight in 10ms buckets: ", delta_times_bucket)

# and convert into a discrete bin of 10ms from midnight
delta_times_bucket_discrete = delta_times_bucket // 10
print("time relative to midnight in 10ms discrete buckets: ", delta_times_bucket_discrete)
print ('')
print (' So we either ruse this bueckt version which tells you which 10ms bucket the frame is in')
print (' or the disrete version which tells you exactly what frame of the video to put the uncompressed video data into')

In [51]:
#####################################
# make some fake video data but don't generate the full array as it will take 100GB of ram
data_cam1_vid1 = np.zeros((frame_times_ms.shape[0]))
print ("video data (ordinarlily this would be an .mp4 that's decompressed on the fly: ", data_cam1_vid1.shape)

# and we crate a 1 min video bucket to hold the videos
stitched_video = np.zeros((18, 60000))
print ("stitched video (18 rpis, 60000 times should be about 10minutes (100fps * 60 * 10)): ", stitched_video.shape)
print (" (the full video is much larger as it has the frames 1024 , 768 and 3 channels for RGB)")

# so now we can loop over the relative to midnight time dicscrete buickets
for i in range(delta_times_bucket_discrete.shape[0]):
    bucket = delta_times_bucket_discrete[i]
    if i%1000==0:
        print("processing frame ", i, " in bucket ", bucket)
    
    # so now we can put the video data into the stitched video
    # this is just a fake example, in reality you would decompress the video and put it into the stitched video
    stitched_video[0, bucket] = data_cam1_vid1[i]  # assuming we are putting cam1 data into the first row


video data (ordinarlily this would be an .mp4 that's decompressed on the fly:  (7205,)
stitched video (18 rpis, 60000 times should be about 10minutes (100fps * 60 * 10)):  (18, 60000)
 (the full video is much larger as it has the frames 1024 , 768 and 3 channels for RGB)
processing frame  0  in bucket  7589
processing frame  1000  in bucket  8422
processing frame  2000  in bucket  9255
processing frame  3000  in bucket  10088
processing frame  4000  in bucket  10921
processing frame  5000  in bucket  11754
processing frame  6000  in bucket  12587
processing frame  7000  in bucket  13420


In [19]:
print("Files:", data.files)
print("First 5 frame_times_ms:", frame_times_ms[:5])
print("Recording start time:", recording_start_time)
print("Encoder start:", encoder_start)
print("Midnight reference:", epoch_start_ms)
print("First timestamp:", frame_times_ms[0])
print("Delta (first):", delta_times[0])


Files: ['frame_times', 'frames_encoded', 'recording_start', 'recording_end', 'encoder_start', 'frame_rate', 'recording_duration', 'recording_mode', 'recording_type', 'recording_duration_continuous', 'recording_duration_triggered', 'recording_start_time']
First 5 frame_times_ms: [1753311675898 1753311675906 1753311675914 1753311675923 1753311675931]
Recording start time: 2025-07-24_00-01-15
Encoder start: 1753311675.907574
Midnight reference: 1753315200000
First timestamp: 1753311675898
Delta (first): -3524102


In [20]:
print("Recording start time (converted):", 
      np.datetime64(int(recording_start_time), 'ms'))


ValueError: invalid literal for int() with base 10: '2025-07-24_00-01-15'